In [1]:
import IPython.display
import sys
import json
from rdflib import *
import glob
import os
import re
from SPARQLWrapper import SPARQLWrapper, JSON, BASIC

In [2]:
def loadAuthFile():
    with open('auth.txt') as f:
        auth =  f.readlines()
        auth = [x.replace('\n','') for x in auth]
    return auth

In [3]:
def getSPARQLResults(queryString, auth, scene):
    result = None
    sparql = SPARQLWrapper(auth[0] + "_event_episode_" + scene)
    sparql.setHTTPAuth(BASIC)
    #sparql.setCredentials(auth[1], auth[2])
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    try :
        json = sparql.query().convert()
        bindings = json['results']['bindings']
        result = bindings
    except  Exception as e:
        print(e)
    return result

In [4]:
scene = "scene4"
auth = loadAuthFile()
query_different_locations = """
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ex: <http://example.org/virtualhome2kg/instance/>
        select distinct ?s ?roomBefore ?roomAfter where { 
            ?s	:action ?action ;
                ho:object ?object ;
                :agent ?agent ;
                :situationBeforeEvent ?before ;
                :situationAfterEvent ?after .
            ?agentBeforeState :isStateOf ?agent ;
                              :partOf ?before ;
                              :bbox ?agentBeforeBbox .
            ?agentAfterState :isStateOf ?agent ;
                             :partOf ?after ;
                             :bbox ?agentAfterBbox .
            ?agentBeforeBbox :inside ?roomBeforeBbox .
            ?agentAfterBbox :inside ?roomAfterBbox .
            ?roomBeforeState :bbox ?roomBeforeBbox ;
                       :isStateOf ?roomBefore ;
                       :partOf ?before .
            ?roomAfterState :bbox ?roomAfterBbox ;
                            :isStateOf ?roomAfter ;
                            :partOf ?after .
            ?roomBefore a [rdfs:subClassOf :Room ] .
            ?roomAfter a [rdfs:subClassOf :Room ] .
            filter (?roomBefore != ?roomAfter)
        }
    """

query_events_and_places = """
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select distinct ?s ?room where { 
            {
                ?s ho:object ?object ;
                    :situationBeforeEvent ?before .
                ?state :isStateOf ?object ;
                       :partOf ?before ;
                       :bbox ?shape .
                ?shape :inside ?roomShape .
                ?roomState :bbox ?roomShape ;
                           :isStateOf ?room .
                ?room a [rdfs:subClassOf :Room] .
            } UNION {
                ?s ho:object ?room .
                ?room a [rdfs:subClassOf :Room] .
            }
        }
    """
json_events_from_to = getSPARQLResults(query_different_locations, auth, scene)
json_events_and_places = getSPARQLResults(query_events_and_places ,auth, scene)

In [5]:
base = Namespace("http://example.org/virtualhome2kg/instance/")
onto = Namespace("http://example.org/virtualhome2kg/ontology/")
events_different_locatins = {}
for result in json_events_from_to:
    event = result["s"]["value"]
    from_location = result["roomBefore"]["value"]
    to_location = result["roomAfter"]["value"]
    events_different_locatins[event] = {"from": from_location, "to": to_location}

g = Graph()
for result in json_events_and_places:
    event_uri = result["s"]["value"]
    place_uri = result["room"]["value"]
    event_r = URIRef(event_uri)
    place_r = URIRef(place_uri)
    if event_uri in events_different_locatins:
        from_location_r = URIRef(events_different_locatins[event_uri]["from"])
        to_location_r = URIRef(events_different_locatins[event_uri]["to"])
        g.add((event_r, onto["from"], from_location_r))
        g.add((event_r, onto["to"], to_location_r))
    else:
        g.add((event_r, onto.place, place_r))

g.bind("ex", base)
g.bind("vh2kg", onto)
output_path = "rdf/episode_" + scene + "/add_places.ttl"
g.serialize(destination=output_path, format="turtle")